In [104]:
import pandas as pd
import os
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import yfinance as yf

In [64]:
symbol = "^VIX"
data_dir = '../data/%s' % symbol
data_file = '%s_history.csv' %symbol;
file_path = os.path.join(data_dir, data_file)

In [65]:
data = pd.read_csv(file_path)

In [66]:
data.head(5)

,Date,Open,High,Low,Close,Volume
0,1990-01-02,17.24,17.24,17.24,17.24,0
1,1990-01-03,18.19,18.19,18.19,18.19,0
2,1990-01-04,19.22,19.22,19.22,19.22,0
3,1990-01-05,20.11,20.11,20.11,20.11,0
4,1990-01-08,20.26,20.26,20.26,20.26,0


In [67]:
price_pt_1 = 40
price_pt_2 = 20
fromDate = '2001-01-01'
toDate = '2021-03-01'
direction = 'UPTREND' if price_pt_1 < price_pt_2 else 'DOWNTREND'

In [68]:
analysis_data_window = data[(data['Date'] >= fromDate) & (data['Date'] <= toDate)]

In [69]:
analysis_data_window.head(5)

,Date,Open,High,Low,Close,Volume
2780,2001-01-02,27.93,30.40,27.92,29.99,0
2781,2001-01-03,29.63,29.96,25.99,26.60,0
2782,2001-01-04,26.35,27.19,26.13,26.97,0
2783,2001-01-05,27.81,29.04,27.21,28.67,0
2784,2001-01-08,29.98,30.80,29.72,29.84,0


In [70]:
analysis_data_window.tail(5)

,Date,Open,High,Low,Close,Volume
7845,2021-02-22,24.46,25.09,21.96,23.45,0
7846,2021-02-23,22.82,27.01,22.50,23.11,0
7847,2021-02-24,23.76,25.04,21.31,21.34,0
7848,2021-02-25,21.73,31.16,21.52,28.89,0
7849,2021-02-26,28.73,30.82,25.23,27.95,0


In [71]:
price_pt_1_all_occurences = analysis_data_window[analysis_data_window['High'] >= price_pt_1]

In [72]:
price_pt_1_all_occurences.head(5)

,Date,Open,High,Low,Close,Volume
2958,2001-09-21,48.93,49.35,42.66,42.66,0
3167,2002-07-23,42.27,46.13,42.05,44.92,0
3168,2002-07-24,48.17,48.46,39.83,39.86,0
3176,2002-08-05,41.95,45.21,41.77,45.08,0
4725,2008-09-29,36.92,48.40,36.92,46.72,0


In [73]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
end_date_obj = datetime.datetime.strptime(toDate, '%Y-%m-%d')

pricePt1Date = []
pricePt2Date = []
calendarDays = []
businessDays = []
pricePt1 = []
pricePt2 = []

while True:
    price_pt_1_date = price_pt_1_all_occurences.iloc[0]['Date']
    filtered_data_set = analysis_data_window[analysis_data_window['Date'] >= price_pt_1_date]
    if direction == 'UPTREND':
        price_pt_2_date = filtered_data_set[filtered_data_set['Close'] >= price_pt_2].iloc[0]['Date']
    else:
        price_pt_2_date = filtered_data_set[filtered_data_set['Close'] <= price_pt_2].iloc[0]['Date']
    
    price_pt_1_date_obj = datetime.datetime.strptime(price_pt_1_date, '%Y-%m-%d')
    pricePt1DateObj.append(price_pt_1_date_obj);
    price_pt_2_date_obj = datetime.datetime.strptime(price_pt_2_date, '%Y-%m-%d')
    pricePt2DateObj.append(price_pt_2_date_obj);
    
    total_cal_days_for_price_change =  (price_pt_2_date_obj - price_pt_1_date_obj).days
    total_business_days_for_price_change = len(pd.date_range(start = price_pt_1_date, end = price_pt_2_date, freq = us_bd))
    
    pricePt1Date.append(price_pt_1_date)
    pricePt2Date.append(price_pt_2_date)
    calendarDays.append(total_cal_days_for_price_change)
    businessDays.append(total_business_days_for_price_change)
    pricePt1.append(price_pt_1)
    pricePt2.append(price_pt_2)
    
    price_pt_1_all_occurences = price_pt_1_all_occurences[price_pt_1_all_occurences['Date'] > price_pt_2_date]
    
    if len(price_pt_1_all_occurences.index) == 0 :
        break
       
        
out_data = {'Price_Pt_1': pricePt1,
        'Price_Pt_1_Date': pricePt1Date, 
        'Price_Pt_2': pricePt2,
        'Price_Pt_2_Date': pricePt2Date,
        'Calendar Days': calendarDays,
        'Business Days': businessDays}
output_df = pd.DataFrame(out_data, columns= ['Price_Pt_1', 'Price_Pt_1_Date', 'Price_Pt_2', 'Price_Pt_2_Date', 'Calendar Days', 'Business Days'])

In [74]:
output_df.head(25)

,Price_Pt_1,Price_Pt_1_Date,Price_Pt_2,Price_Pt_2_Date,Calendar Days,Business Days
0,45,2001-09-21,25,2001-11-19,59,40
1,45,2002-07-23,25,2002-11-21,121,85
2,45,2008-09-29,25,2009-07-17,291,201
3,45,2010-05-20,25,2010-06-18,29,21
4,45,2011-08-08,25,2011-10-28,81,58
5,45,2015-08-24,25,2015-09-08,15,11
6,45,2018-02-06,25,2018-02-13,7,6
7,45,2020-02-28,25,2020-06-05,98,70


In [81]:
pricePt1Date

['2001-09-21',
 '2002-07-23',
 '2008-09-29',
 '2010-05-20',
 '2011-08-08',
 '2015-08-24',
 '2018-02-06',
 '2020-02-28']

In [83]:
dates_of_interest = pd.Series(pricePt1Date)
dates_of_interest

0    2001-09-21
1    2002-07-23
2    2008-09-29
3    2010-05-20
4    2011-08-08
5    2015-08-24
6    2018-02-06
7    2020-02-28
dtype: object

In [94]:
dates_of_interest = dates_of_interest.apply(lambda x: (( datetime.datetime.strptime(x, '%Y-%m-%d') - datetime.timedelta(days=60)).strftime('%Y-%m-%d'))  )

In [95]:
dates_of_interest

0    2001-07-23
1    2002-05-24
2    2008-07-31
3    2010-03-21
4    2011-06-09
5    2015-06-25
6    2017-12-08
7    2019-12-30
dtype: object

In [108]:
pricePt2Date

['2001-11-19',
 '2002-11-21',
 '2009-07-17',
 '2010-06-18',
 '2011-10-28',
 '2015-09-08',
 '2018-02-13',
 '2020-06-05']

In [161]:
all_data_df = pd.DataFrame([])
for items in dates_of_interest.iteritems():
    date_1 = items[1]
    date_2 = pricePt2Date[items[0]]
    data_of_interest = yf.download(tickers = '^VIX SPY VXX', start = date_1, end = date_2, group_by = 'ticker', auto_adjust  = True)
    data_of_interest['Study_Item_Num'] = [(items[0]+1) for _ in range(len(data_of_interest))]
    all_data_df = all_data_df.append(data_of_interest);

[*********************100%***********************]  3 of 3 completed

1 Failed download:
- VXX: Data doesn't exist for startDate = 995860800, endDate = 1006146000
[*********************100%***********************]  3 of 3 completed

1 Failed download:
- VXX: Data doesn't exist for startDate = 1022212800, endDate = 1037854800
[*********************100%***********************]  3 of 3 completed

1 Failed download:
- VXX: Data doesn't exist for startDate = 1217476800, endDate = 1247803200
[*********************100%***********************]  3 of 3 completed

1 Failed download:
- VXX: Data doesn't exist for startDate = 1269144000, endDate = 1276833600
[*********************100%***********************]  3 of 3 completed

1 Failed download:
- VXX: Data doesn't exist for startDate = 1307592000, endDate = 1319774400
[*********************100%***********************]  3 of 3 completed

1 Failed download:
- VXX: Data doesn't exist for startDate = 1435204800, endDate = 1441684800
[****************

In [154]:
len(all_data_df.index)

814

In [162]:
all_data_df.head(1000)

SPY                                                   \
                 Close        High         Low        Open       Volume   
Date                                                                      
2001-07-23   81.565338   83.574472   81.565338   83.519619    8065200.0   
2001-07-24   80.776764   81.736755   80.056765   81.599615   12269000.0   
2001-07-25   81.668205   81.928779   80.543639   80.859066   12088500.0   
2001-07-26   82.525360   82.868216   81.297936   81.640792   12898200.0   
2001-07-27   82.840767   83.211052   82.223631   82.854484    8478800.0   
...                ...         ...         ...         ...          ...   
2020-05-29  300.497772  301.129718  295.708682  298.661118  119265700.0   
2020-06-01  301.712280  302.363994  299.253564  299.806528   55758300.0   
2020-06-02  304.210480  304.259870  301.267928  302.699698   74267200.0   
2020-06-03  308.259033  309.285979  306.047177  306.343397   92567600.0   
2020-06-04  307.449310  309.068726  305.197948  307.202450   75794400.0   

           Study_Item_Num       VXX                                   \
                          Adj Close      Close       High        Low   
Date                                                                   
2001-07-23              1       NaN        NaN        NaN        NaN   
2001-07-24              1       NaN        NaN        NaN        NaN   
2001-07-25              1       NaN        NaN        NaN        NaN   
2001-07-26              1       NaN        NaN        NaN        NaN   
2001-07-27              1       NaN        NaN        NaN        NaN   
...                   ...       ...        ...        ...        ...   
2020-05-29              8       NaN  33.130001  35.290001  33.130001   
2020-06-01              8       NaN  33.419998  34.189999  33.230000   
2020-06-02              8       NaN  32.490002  33.653000  32.459999   
2020-06-03              8       NaN  31.090000  32.040001  30.940001   
2020-06-04              8       NaN  30.840000  31.580000  29.930000   

                                        ^VIX                                   \
                 Open      Volume      Close       High        Low       Open   
Date                                                                            
2001-07-23        NaN         NaN  23.740000  23.870001  22.510000  22.639999   
2001-07-24        NaN         NaN  25.240000  25.610001  24.160000  24.200001   
2001-07-25        NaN         NaN  24.000000  25.230000  23.920000  24.920000   
2001-07-26        NaN         NaN  23.010000  24.230000  22.980000  24.020000   
2001-07-27        NaN         NaN  22.000000  23.129999  21.930000  23.020000   
...               ...         ...        ...        ...        ...        ...   
2020-05-29  34.660000  32572200.0  27.510000  30.160000  27.290001  29.299999   
2020-06-01  34.060001  15480100.0  28.230000  30.600000  28.110001  28.940001   
2020-06-02  33.220001  22796000.0  26.840000  28.520000  26.660000  28.400000   
2020-06-03  31.879999  24819800.0  25.660000  26.980000  25.040001  26.750000   
2020-06-04  31.290001  28998500.0  25.809999  26.430000  24.379999  26.230000   

                   
           Volume  
Date               
2001-07-23    0.0  
2001-07-24    0.0  
2001-07-25    0.0  
2001-07-26    0.0  
2001-07-27    0.0  
...           ...  
2020-05-29    0.0  
2020-06-01    0.0  
2020-06-02    0.0  
2020-06-03    0.0  
2020-06-04    0.0  

[814 rows x 17 columns]

In [164]:
output_dir = '../data/vix_analysis'
if not os.path.exists(output_dir):
    os.mkdir(output_dir);
    
output_file = 'data.xlsx';

file_path = os.path.join(output_dir, output_file)

all_data_df.to_excel(file_path)